In [1]:
import datetime
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import numpy as np
import os, sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QLineEdit
from PyQt5.QtCore import QTimer

In [2]:
L, page = np.zeros(1), 1

class App(QMainWindow):
    def __init__(self):
        super().__init__()
        self.loadnpy()
        self.initUI()
        self.show()
    def loadnpy(self):
        global L
        for data in os.listdir():
            if data[:3] == "raw": 
                try:
                    file = np.load(data, allow_pickle=True)
                    L = np.concatenate((L,file))
                except:
                    print("dimension error at {}".format(data))
        L = (L[1:]-L[1:].mean())/L[1:].std()
        self.max_data, self.max_page = len(L), int(np.ceil(len(L)//200))
    def initUI(self):
        global page
        self.setWindowTitle('Select vibration signal')
        self.setGeometry(100, 200, 1440, 650)
        
        obj = PlotCanvas(self, width=15, height=4)
        self.timer = QTimer(obj)
        self.timer.timeout.connect(obj.plotc)
        self.FRAME_PER_SECOND = 10
        self.timer.start(1000/self.FRAME_PER_SECOND)
        
        self.button_prev = QPushButton(self)
        self.button_prev.setText("prev page")
        self.button_prev.move(100, 500)
        self.button_prev.resize(100, 50)
        self.button_prev.clicked.connect(self.detect_prev)
        
        self.label_page = QLabel(self)
        self.label_page.setText("page={}/{}\ntotal data={}".format(page, \
            self.max_page, self.max_data))
        self.label_page.move(250, 475)
        self.label_page.resize(150, 100)
        
        self.button_next = QPushButton(self)
        self.button_next.setText("next page")
        self.button_next.move(400, 500)
        self.button_next.resize(100, 50)
        self.button_next.clicked.connect(self.detect_next)
        
        self.label_init = QLabel(self)
        self.label_init.setText("init  =")
        self.label_init.move(545, 410)
        self.label_init.resize(100, 30)
        self.label_start = QLabel(self)
        self.label_start.setText("start =")
        self.label_start.move(545, 460)
        self.label_start.resize(100, 30)
        self.label_die = QLabel(self)
        self.label_die.setText("die   =")
        self.label_die.move(545, 510)
        self.label_die.resize(100, 30)
        self.label_end = QLabel(self)
        self.label_end.setText("end   =")
        self.label_end.move(545, 560)
        self.label_end.resize(100, 30)
        self.label_final = QLabel(self)
        self.label_final.setText("final =")
        self.label_final.move(545, 610)
        self.label_final.resize(100, 30)
        
        self.line_init = QLineEdit(self)
        self.line_init.move(600, 410)
        self.line_init.resize(600, 30)
        self.line_start = QLineEdit(self)
        self.line_start.move(600, 460)
        self.line_start.resize(600, 30)
        self.line_die = QLineEdit(self)
        self.line_die.move(600, 510)
        self.line_die.resize(600, 30)
        self.line_end = QLineEdit(self)
        self.line_end.move(600, 560)
        self.line_end.resize(600, 30)
        self.line_final = QLineEdit(self)
        self.line_final.move(600, 610)
        self.line_final.resize(600, 30)
        
        self.button_save = QPushButton(self)
        self.button_save.setText("save")
        self.button_save.move(1210, 500)
        self.button_save.resize(100, 50)
        self.button_save.clicked.connect(self.save)
        
    def detect_prev(self):
        global page, L
        if page>1:
            page-=1
            self.label_page.setText("page={}/{}\ntotal data={}".format(page, \
                self.max_page, self.max_data))
            
    def detect_next(self):
        global page, L
        if page<np.ceil(len(L)//200):
            page+=1
            self.label_page.setText("page={}/{}\ntotal data={}".format(page, \
                self.max_page, self.max_data)) 
            
    def save(self):
        init, start, die, end, final = self.line_init.text(), self.line_start.text(), \
            self.line_die.text(), self.line_end.text(), self.line_final.text()
        alist = [init.split(" "), start.split(" "), die.split(" "), end.split(" "), \
            final.split(" ")]
        cut = np.zeros((5,len(alist[0])))
        for i in range(5):
            for j in range(len(alist[0])):
                cut[i][j] = int(alist[i][j])
        np.save('cut.npy', cut)
        
class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None, width=15, height=4, dpi=100):
        super().__init__( Figure(figsize=(width, height), dpi=dpi) )
        self.setParent(parent)
        self.plotc()
        
    def plotc(self):
        global L, page
        fig = self.figure.add_subplot(1,1,1)
        fig.clear()
        if len(L)>200:
            if page==np.ceil(len(L)//200):
                fig.plot(np.arange(200*(page-1), 200*(page-1)+len(L[200*(page-1):])),\
                    L[200*(page-1):], 'r-')
            else:
                fig.plot(np.arange(200*(page-1),200*page), L[200*(page-1):200*page], 'r-')
        else:
            fig.plot(L, 'r-')
        fig.set_ylim(-2,2)
        fig.set_title('Vibration')
        fig.axes.grid(True)
        self.draw()

In [3]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:125: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


SystemExit: 0

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
